# Import libraries


In [1]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt
#from wordcloud import WordCloud
from math import log, sqrt
import pandas as pd
import numpy as np
import re
%matplotlib inline
import string

# Retrieve Mail

In [2]:
emails = pd.read_csv('spam.csv', encoding = 'latin-1', )


# Preprocessing

In [3]:
emails.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis = 1, inplace = True)


In [4]:
emails.rename(columns = {'v1': 'spam/ham', 'v2': 'email message'}, inplace = True)
emails.head()

,spam/ham,email message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
emails['spam/ham'].value_counts()

ham     4825
spam     747
Name: spam/ham, dtype: int64

In [6]:
emails['spam/ham'] = emails['spam/ham'].map({'ham': 0, 'spam': 1})
emails.head()

,spam/ham,email message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [7]:
emails.shape[0]

5572

# splitting

In [8]:
totalMails = 5572
trainIndex, testIndex = list(), list()
for i in range(emails.shape[0]):
    if np.random.uniform(0, 1) < 0.80:
        trainIndex += [i]
    else:
        testIndex += [i]
trainData = emails.loc[trainIndex]
testData = emails.loc[testIndex]

In [9]:
trainData = trainData.reset_index()
testData = testData.reset_index()

In [10]:
trainData['spam/ham'].value_counts()

0    3854
1     574
Name: spam/ham, dtype: int64

In [11]:
testData['spam/ham'].value_counts()

0    971
1    173
Name: spam/ham, dtype: int64

In [12]:
#spam_words = ' '.join(list(emails[emails['spam/ham'] == 1]['email message']))
#spam_wc = WordCloud(width = 512,height = 512).generate(spam_words)
#plt.figure(figsize = (10, 8), facecolor = 'k')
#plt.imshow(spam_wc)
#plt.axis('off')
#plt.tight_layout(pad = 0)
#plt.show()

In [13]:
#ham_words = ' '.join(list(emails[emails['spam/ham'] == 0]['email message']))
#ham_wc = WordCloud(width = 512,height = 512).generate(ham_words)
#plt.figure(figsize = (10, 8), facecolor = 'k')
#plt.imshow(ham_wc)
#plt.axis('off')
#plt.tight_layout(pad = 0)
#plt.show()

In [14]:
X = np.array(trainData)
len(X)

4428

# Text Pre Processing

In [18]:
def text_process(mess):
    mess = mess.lower()
    nopunc =[char for char in mess if char not in string.punctuation]
    nopunc=''.join(nopunc)
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [19]:
processed_trainData = trainData['email message'].apply(text_process)

In [20]:
testData.head()

,index,spam/ham,email message
0,3,0,U dun say so early hor... U c already then say...
1,13,0,I've been searching for the right words to tha...
2,15,1,"XXXMobileMovieClub: To use your credit, click ..."
3,17,0,Eh u remember how 2 spell his name... Yes i di...
4,33,0,For fear of fainting with the of all that hous...


# Dictionary

In [21]:
def dictionary_func(mess, label):
    for i in range(len(mess)):
        if mess[i] in dictionary:
            count[dictionary.index(mess[i])] = count[dictionary.index(mess[i])]  + 1
            if(label == 1):
                spam[dictionary.index(mess[i])] = spam[dictionary.index(mess[i])] + 1
                
            else:
                ham[dictionary.index(mess[i])] = ham[dictionary.index(mess[i])] + 1
        else: 
            dictionary.append(mess[i])
            count.append(1)
            if(label == 1):
                spam.append(1)
                ham.append(0)
            else:
                ham.append(1)
                spam.append(0)
    

In [22]:
dictionary = []
count = []
spam = []
ham = []

for i in range(len(processed_trainData)):
    X = trainData.iloc[i]
    label = X[1]
    dictionary_func(processed_trainData[i],label)
print(len(count))
print(len(ham))
print(len(spam))
print(len(dictionary))

8165
8165
8165
8165


In [23]:
X = trainData['spam/ham'].value_counts()
count_of_spam = X[1]
count_of_ham = X[0]

In [24]:
probability_of_spam = float(count_of_spam)/ len(trainData)
probability_of_ham = float(count_of_ham)/ len(trainData)

In [25]:
probability_of_ham

0.8703703703703703

In [26]:
total_words_in_spam = np.count_nonzero(spam)
total_words_in_ham = np.count_nonzero(ham)

# Probabiity of Word in Dictionary

In [27]:
probability_of_words = []
for i in range(len(dictionary)):
    total_occurance_of_word_in_train = spam[i] + ham[i]
    probability_of_word = float(total_occurance_of_word_in_train)/float(len(dictionary))
    probability_of_words.append(probability_of_word)

# Probability of a word in spam

In [28]:
def func_probability_of_each_word_in_spam(processed_message):
    probability_of_each_word_in_spam = []
    probability_of_each_word_in_ham = []
    #print(processed_message.shape)
    for i in range(len(processed_message)):
        if processed_message[i] in dictionary:
            #print("processed_message[i]",processed_message[i])
            if spam[dictionary.index(processed_message[i])] != 0:
                probability_of_word_in_spam = (float(spam[(dictionary.index(processed_message[i]))])) / (float(total_words_in_spam))
                
            else:
                probability_of_word_in_spam = (1.0)/  float(sum(spam))
        else:
            #print("Not present")
            probability_of_word_in_spam = (1.0)/ float(sum(spam) +sum(ham))
        probability_of_each_word_in_spam.append(probability_of_word_in_spam)
        #print(dictionary[dictionary.index(processed_message[i])],":",probability_of_word_in_spam)
        #print(len(probability_of_each_word_in_spam))
    return probability_of_each_word_in_spam

# Probability of a word in ham

In [29]:
def func_probability_of_each_word_in_ham(processed_message):
    probability_of_each_word_in_spam = []
    probability_of_each_word_in_ham = []
    #print(processed_message.shape)
    alpha = 1
    for i in range(len(processed_message)):
        if processed_message[i] in dictionary:
            #print("processed_message[i]",processed_message[i])
            if ham[dictionary.index(processed_message[i])] != 0:
                probability_of_word_in_ham = (float(ham[(dictionary.index(processed_message[i]))])) / (float(total_words_in_ham))
                
            else:
                #print("Not present")
                probability_of_word_in_ham = (1.0)/  float(sum(ham))
                
        else:
            #print("Not present")
            probability_of_word_in_ham = (1.0)/ float(sum(spam) +sum(ham))
        #print(dictionary[dictionary.index(processed_message[i])],":",probability_of_word_in_ham)
        probability_of_each_word_in_ham.append(probability_of_word_in_ham)
    return probability_of_each_word_in_ham

# Probability of spam for a message

In [30]:
def probability_of_spam_for_a_message(probability_of_each_word_in_spam):
    probability_of_spam_with_given_word = 1
    for i in range( len(probability_of_each_word_in_spam)):
        #print("word with given spam")
        #print(probability_of_each_word_in_spam[i])
        probability_of_spam_with_given_word *= probability_of_each_word_in_spam[i]
   # print("spam with given word part1")
   # print(probability_of_spam_with_given_word)
    probability_of_spam_with_given_word *= probability_of_spam
   # print("spam with given word part2")
   # print(probability_of_spam_with_given_word)
    return probability_of_spam_with_given_word

# probability of ham for a message

In [31]:
def probability_of_ham_for_a_message(probability_of_each_word_in_ham):
    probability_of_ham_with_given_word = 1
    for i in range( len(probability_of_each_word_in_ham)):
        
        #print("word with given spam")
        #print(probability_of_each_word_in_spam[i])
        probability_of_ham_with_given_word *= probability_of_each_word_in_ham[i]
    #print("ham with given word part1")
   # print(probability_of_ham_with_given_word)
    probability_of_ham_with_given_word = probability_of_ham_with_given_word * probability_of_ham
   # print("ham with given word part2")
    #print(probability_of_ham_with_given_word)
    return probability_of_ham_with_given_word

In [34]:
message = "Heyy my name is Diksha"
processed_message = text_process(message)
(processed_message)

['heyy', 'name', 'diksha']

In [35]:
X = trainData.iloc[:,2]

# Training 

In [36]:
probability_spam = []
probability_ham  = []
for i in range(len(trainData)):
    processed_message = text_process(X[i])
    probability_of_each_word_in_spam = func_probability_of_each_word_in_spam(processed_message)
    A = probability_of_spam_for_a_message(probability_of_each_word_in_spam)
    probability_spam.append(A)
    probability_of_each_word_in_ham = func_probability_of_each_word_in_ham(processed_message)
    B = probability_of_ham_for_a_message(probability_of_each_word_in_ham)
    probability_ham.append(B)                         

# classification and Prediction

In [37]:
spam_or_ham = []
for i in range(len(trainData)):
    if probability_ham[i] > probability_spam[i]:
        spam_or_ham.append(0)
    else:
        spam_or_ham.append(1)

In [38]:
target = trainData.iloc[:,1]
target = np.array(target)
predicted = (spam_or_ham)
predicted = np.array(predicted)

x = predicted[0]
predicted.dtype
predicted = predicted.astype(np.int)


# Accuracy on Training Data

In [39]:
def Accuracy(p,Data):
    count = 0
    for i in range(len(Data)):
        if p[i]== target[i]:
            count = count +1
    
    return float(count)/float(len(Data)) * 100.0

In [40]:
accuracy = Accuracy(predicted, trainData)
accuracy

99.52574525745257

# On Test Data

In [41]:
Y = testData.iloc[:,2]

In [42]:
probability_spam_test = []
probability_ham_test = []
for i in range(0,len(testData)):
    processed_message = text_process(Y[i])
    probability_of_each_word_in_spam = func_probability_of_each_word_in_spam(processed_message)
    A = probability_of_spam_for_a_message(probability_of_each_word_in_spam)
    probability_spam_test.append(A)
    probability_of_each_word_in_ham = func_probability_of_each_word_in_ham(processed_message)
    B = probability_of_ham_for_a_message(probability_of_each_word_in_ham)
    probability_ham_test.append(B)                         

In [43]:
spam_or_ham_test = []
for i in range(len(testData)):
    if probability_ham_test[i] > probability_spam_test[i]:
        spam_or_ham_test.append(0)
    else:
        spam_or_ham_test.append(1)

In [44]:
target = testData.iloc[:,1]
target = np.array(target)
predicted_test = (spam_or_ham_test)
predicted_test = np.array(predicted)

x = predicted_test[0]
predicted_test.dtype
predicted_test = predicted_test.astype(np.int)


# Accuracy on Test Data

In [45]:
accuracy = Accuracy(predicted_test,testData)
accuracy

74.12587412587412